# Module 7: Contours & Shape Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/07_Module.ipynb)

**Week 11-12: Contour Detection, Shape Properties, Bounding Boxes, Hough Transform**

## Learning Objectives
- Understand contour detection and representation
- Analyze shape properties (area, perimeter, moments)
- Apply shape approximation and fitting algorithms
- Use Hough Transform for line and circle detection
- Build practical shape analysis applications

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Contours and Shapes

### What is a Contour?

A **contour** is a curve joining all continuous points along a boundary having the same intensity.

**Formal definition**: A contour is a sequence of points:
$$
C = \{(x_0, y_0), (x_1, y_1), \ldots, (x_{n-1}, y_{n-1})\}
$$

### Why Contours?

1. **Shape analysis**: Describe object boundaries
2. **Object detection**: Find and classify objects
3. **Object recognition**: Match shapes
4. **Measurement**: Compute area, perimeter, dimensions
5. **Tracking**: Follow objects over time

### Contour Properties

**Geometric**:
- Area, perimeter, circularity
- Centroid, orientation
- Convexity, solidity

**Topological**:
- Hierarchy (parent-child relationships)
- Holes (inner contours)
- Connectivity

In [ ]:
# Create test image with various shapes
print("=" * 70)
print("CREATING TEST IMAGE WITH SHAPES")
print("=" * 70)

# Create blank image
img = np.zeros((500, 600, 3), dtype=np.uint8)
img[:] = (255, 255, 255)  # White background

# Draw various shapes
# Rectangle
cv2.rectangle(img, (50, 50), (150, 150), (255, 0, 0), -1)

# Circle
cv2.circle(img, (250, 100), 50, (0, 255, 0), -1)

# Triangle
triangle = np.array([[400, 50], [350, 150], [450, 150]], dtype=np.int32)
cv2.fillPoly(img, [triangle], (0, 0, 255))

# Pentagon
pentagon = np.array([
    [100, 250], [50, 320], [75, 400], [125, 400], [150, 320]
], dtype=np.int32)
cv2.fillPoly(img, [pentagon], (255, 255, 0))

# Star
def create_star(center, outer_r, inner_r, n_points=5):
    points = []
    angle = np.pi / n_points
    for i in range(2 * n_points):
        r = outer_r if i % 2 == 0 else inner_r
        theta = i * angle - np.pi / 2
        x = int(center[0] + r * np.cos(theta))
        y = int(center[1] + r * np.sin(theta))
        points.append([x, y])
    return np.array(points, dtype=np.int32)

star = create_star((300, 325), 60, 25)
cv2.fillPoly(img, [star], (255, 0, 255))

# Ellipse
cv2.ellipse(img, (500, 325), (60, 40), 30, 0, 360, (0, 255, 255), -1)

print("\nShapes created:")
print("  - Rectangle (blue)")
print("  - Circle (green)")
print("  - Triangle (red)")
print("  - Pentagon (yellow)")
print("  - Star (magenta)")
print("  - Ellipse (cyan)")

plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Test Image with Various Shapes')
plt.axis('off')
plt.show()

### 1. Finding Contours

OpenCV's `findContours()` uses the **border following** algorithm.

#### Algorithm

1. Scan image from top-left to find first boundary pixel
2. Follow the boundary using **8-connectivity** or **4-connectivity**
3. Store boundary points
4. Continue until back at starting point
5. Mark processed region
6. Repeat for next unprocessed boundary

#### Retrieval Modes

**`RETR_EXTERNAL`**: Only outermost contours
$$
\text{Hierarchy: } [\text{Next}, \text{Previous}, -1, -1]
$$

**`RETR_LIST`**: All contours (no hierarchy)
$$
\text{Hierarchy: } [\text{Next}, \text{Previous}, -1, -1]
$$

**`RETR_TREE`**: Full hierarchy (parent-child)
$$
\text{Hierarchy: } [\text{Next}, \text{Previous}, \text{First_Child}, \text{Parent}]
$$

**`RETR_CCOMP`**: Two-level hierarchy
$$
\text{Level 1: Outer boundaries} \\
\text{Level 2: Hole boundaries}
$$

#### Approximation Methods

**`CHAIN_APPROX_NONE`**: Store all boundary points
- Most accurate
- Large memory

**`CHAIN_APPROX_SIMPLE`**: Compress horizontal, vertical, diagonal segments
- Remove redundant points
- Efficient storage

**Example**: Rectangle
- `NONE`: Stores hundreds of points
- `SIMPLE`: Stores only 4 corner points!

In [ ]:
# Find contours
print("=" * 70)
print("FINDING CONTOURS")
print("=" * 70)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold to binary
_, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

# Find contours
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"\nNumber of contours found: {len(contours)}")
print(f"Hierarchy shape: {hierarchy.shape if hierarchy is not None else None}")

# Analyze each contour
print(f"\nContour details:")
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    num_points = len(contour)
    print(f"  Contour {i}: {num_points} points, area={area:.0f}, perimeter={perimeter:.2f}")

# Draw contours
img_contours = img.copy()
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 3)

# Draw contours with different colors
img_colored = img.copy()
for i, contour in enumerate(contours):
    color = tuple(np.random.randint(0, 255, 3).tolist())
    cv2.drawContours(img_colored, [contour], 0, color, 3)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(binary, cmap='gray')
axes[0, 1].set_title('Binary Image')
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(img_contours, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title(f'All Contours (Green)\n{len(contours)} contours')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_colored, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Colored Contours\n(Each shape different color)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Contours represent object boundaries!")

### 2. Image Moments

**Moments** are weighted averages of pixel intensities used to describe shape properties.

#### Raw Moments

**Definition**:
$$
M_{ij} = \sum_x \sum_y x^i y^j I(x, y)
$$

**Special cases**:
- $M_{00}$: Total mass (area for binary images)
- $M_{10}$, $M_{01}$: First moments (used for centroid)

#### Centroid (Center of Mass)

$$
\bar{x} = \frac{M_{10}}{M_{00}}, \quad \bar{y} = \frac{M_{01}}{M_{00}}
$$

#### Central Moments

Moments calculated relative to centroid:
$$
\mu_{ij} = \sum_x \sum_y (x - \bar{x})^i (y - \bar{y})^j I(x, y)
$$

**Properties**:
- Translation invariant
- $\mu_{20}$, $\mu_{02}$, $\mu_{11}$: Second-order moments (orientation, elongation)

#### Hu Moments

**Normalized central moments**:
$$
\eta_{ij} = \frac{\mu_{ij}}{\mu_{00}^{1 + (i+j)/2}}
$$

**Hu's 7 invariant moments** (translation, scale, rotation invariant):
$$
h_1 = \eta_{20} + \eta_{02}
$$
$$
h_2 = (\eta_{20} - \eta_{02})^2 + 4\eta_{11}^2
$$
$$
\vdots
$$

**Applications**: Shape matching, object recognition

In [ ]:
# Compute moments and shape properties
print("=" * 70)
print("IMAGE MOMENTS AND SHAPE PROPERTIES")
print("=" * 70)

img_analysis = img.copy()

print(f"\nAnalyzing {len(contours)} shapes:\n")

for i, contour in enumerate(contours):
    # Compute moments
    M = cv2.moments(contour)
    
    # Calculate centroid
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    
    # Geometric properties
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # Compactness (circularity)
    if perimeter > 0:
        compactness = 4 * np.pi * area / (perimeter ** 2)
    else:
        compactness = 0
    
    # Hu moments
    hu_moments = cv2.HuMoments(M).flatten()
    
    print(f"Shape {i}:")
    print(f"  Centroid: ({cx}, {cy})")
    print(f"  Area: {area:.2f}")
    print(f"  Perimeter: {perimeter:.2f}")
    print(f"  Compactness: {compactness:.4f} (1.0 = perfect circle)")
    print(f"  Hu Moments: [{', '.join([f'{h:.2e}' for h in hu_moments[:3]])}...]")
    print()
    
    # Draw centroid
    cv2.circle(img_analysis, (cx, cy), 5, (255, 0, 0), -1)
    cv2.putText(img_analysis, f'{i}', (cx + 10, cy), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

# Visualize
plt.figure(figsize=(12, 10))
plt.imshow(cv2.cvtColor(img_analysis, cv2.COLOR_BGR2RGB))
plt.title('Shape Analysis\n(Blue dots = centroids)')
plt.axis('off')
plt.show()

print("Key Insight: Moments provide quantitative shape descriptors!")

### 3. Bounding Shapes

**Bounding shapes** enclose contours for simplified representation.

#### Bounding Rectangle (Axis-Aligned)

$$
(x, y, w, h) = (\min x_i, \min y_i, \max x_i - \min x_i, \max y_i - \min y_i)
$$

- Fast to compute
- Axis-aligned (not rotation-invariant)

#### Minimum Area Rectangle (Rotated)

Finds smallest rectangle enclosing contour (can be rotated):
$$
\text{MinAreaRect} = ((c_x, c_y), (w, h), \theta)
$$

Where:
- $(c_x, c_y)$: Center
- $(w, h)$: Width and height
- $\theta$: Rotation angle

#### Minimum Enclosing Circle

$$
((c_x, c_y), r) \text{ where } r = \min_{c} \max_i \|p_i - c\|
$$

#### Fitting Ellipse

Fit ellipse using least squares:
$$
\frac{(x-c_x)^2}{a^2} + \frac{(y-c_y)^2}{b^2} = 1
$$

**Requires**: At least 5 points

In [ ]:
# Bounding shapes
print("=" * 70)
print("BOUNDING SHAPES")
print("=" * 70)

# Create images for different bounding shapes
img_rect = img.copy()
img_min_rect = img.copy()
img_circle = img.copy()
img_ellipse = img.copy()

for i, contour in enumerate(contours):
    # 1. Bounding rectangle (axis-aligned)
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(img_rect, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # 2. Minimum area rectangle (rotated)
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    cv2.drawContours(img_min_rect, [box], 0, (255, 0, 0), 2)
    
    # 3. Minimum enclosing circle
    (cx, cy), radius = cv2.minEnclosingCircle(contour)
    center = (int(cx), int(cy))
    radius = int(radius)
    cv2.circle(img_circle, center, radius, (0, 0, 255), 2)
    
    # 4. Fitting ellipse (requires at least 5 points)
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        cv2.ellipse(img_ellipse, ellipse, (255, 0, 255), 2)

print(f"\nBounding shapes computed for {len(contours)} contours")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

axes[0, 0].imshow(cv2.cvtColor(img_rect, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Bounding Rectangle\n(Axis-aligned, Green)')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img_min_rect, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Minimum Area Rectangle\n(Rotated, Blue)')
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(img_circle, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Minimum Enclosing Circle\n(Red)')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_ellipse, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Fitted Ellipse\n(Magenta)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Different bounding shapes for different purposes!")

### 4. Contour Approximation

**Contour approximation** simplifies contours by reducing the number of points.

#### Douglas-Peucker Algorithm

`cv2.approxPolyDP()` uses the **Douglas-Peucker** algorithm:

**Algorithm**:
1. Draw line from first to last point
2. Find point with maximum distance $d$ from line
3. If $d > \epsilon$: Split at that point, recurse
4. If $d \leq \epsilon$: Approximate with straight line

**Parameter** $\epsilon$:
- Maximum distance from original contour
- Typically: $\epsilon = k \cdot \text{perimeter}$ where $k \in [0.01, 0.05]$
- Larger $\epsilon$ → fewer points (simpler shape)

#### Convex Hull

**Convex hull** is the smallest convex set containing all contour points.

**Mathematical definition**:
$$
\text{ConvexHull}(S) = \left\{ \sum_{i} \lambda_i p_i : p_i \in S, \lambda_i \geq 0, \sum_i \lambda_i = 1 \right\}
$$

**Convexity defects**: Deviations from convex hull
- Used for gesture recognition (e.g., counting fingers)
- Depth: Maximum distance from hull

In [ ]:
# Contour approximation
print("=" * 70)
print("CONTOUR APPROXIMATION")
print("=" * 70)

# Use the star contour for demonstration
star_contour = contours[4]  # Star is typically the 5th shape

# Different epsilon values
epsilons = [0.001, 0.01, 0.05, 0.1]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

print(f"\nOriginal contour points: {len(star_contour)}\n")

for idx, epsilon_factor in enumerate(epsilons):
    # Approximate contour
    perimeter = cv2.arcLength(star_contour, True)
    epsilon = epsilon_factor * perimeter
    approx = cv2.approxPolyDP(star_contour, epsilon, True)
    
    print(f"ε = {epsilon_factor} × perimeter:")
    print(f"  Approximated points: {len(approx)}")
    print(f"  Reduction: {100 * (1 - len(approx) / len(star_contour)):.1f}%")
    
    # Draw
    img_approx = np.ones((300, 300, 3), dtype=np.uint8) * 255
    # Scale and center the contour
    scaled_original = ((star_contour - star_contour.min(axis=0)) / 
                       (star_contour.max(axis=0) - star_contour.min(axis=0)) * 200 + 50).astype(np.int32)
    scaled_approx = ((approx - star_contour.min(axis=0)) / 
                     (star_contour.max(axis=0) - star_contour.min(axis=0)) * 200 + 50).astype(np.int32)
    
    cv2.drawContours(img_approx, [scaled_original], -1, (200, 200, 200), 1)
    cv2.drawContours(img_approx, [scaled_approx], -1, (255, 0, 0), 3)
    
    # Mark approximation points
    for point in scaled_approx:
        cv2.circle(img_approx, tuple(point[0]), 5, (0, 255, 0), -1)
    
    axes[idx].imshow(cv2.cvtColor(img_approx, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'ε = {epsilon_factor}\n{len(approx)} points')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Convex hull
hull = cv2.convexHull(star_contour)
img_hull = np.ones((300, 300, 3), dtype=np.uint8) * 255

scaled_star = ((star_contour - star_contour.min(axis=0)) / 
               (star_contour.max(axis=0) - star_contour.min(axis=0)) * 200 + 50).astype(np.int32)
scaled_hull = ((hull - star_contour.min(axis=0)) / 
               (star_contour.max(axis=0) - star_contour.min(axis=0)) * 200 + 50).astype(np.int32)

cv2.fillPoly(img_hull, [scaled_star], (255, 200, 200))
cv2.drawContours(img_hull, [scaled_hull], -1, (0, 0, 255), 3)
cv2.drawContours(img_hull, [scaled_star], -1, (255, 0, 0), 2)

plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(img_hull, cv2.COLOR_BGR2RGB))
plt.title('Convex Hull (Red)\nOriginal Contour (Blue)')
plt.axis('off')
plt.show()

print(f"\nConvex hull points: {len(hull)}")
print("\nKey Insight: Approximation simplifies contours while preserving shape!")

### 5. Hough Transform

The **Hough Transform** detects parametric shapes (lines, circles) in images.

#### Hough Line Transform

**Problem**: Find lines in edge images

**Line representation** (polar coordinates):
$$
\rho = x \cos\theta + y \sin\theta
$$

Where:
- $\rho$: Distance from origin to line
- $\theta$: Angle of perpendicular from origin

**Algorithm**:
1. Create **accumulator** array $A(\rho, \theta)$
2. For each edge pixel $(x, y)$:
   - For each $\theta \in [0, \pi]$:
     - Compute $\rho = x\cos\theta + y\sin\theta$
     - Increment $A(\rho, \theta)$
3. Find peaks in accumulator (lines!)

**Intuition**: All lines through $(x, y)$ form a sinusoid in $\rho$-$\theta$ space.
Intersection of sinusoids → collinear points → line!

#### Probabilistic Hough Transform

**Improvement**: Use random subset of edge pixels
- Much faster
- Detects line segments (with endpoints)
- Returns: $(x_1, y_1, x_2, y_2)$

#### Hough Circle Transform

**Circle equation**:
$$
(x - a)^2 + (y - b)^2 = r^2
$$

**Parameters**: $(a, b, r)$ (center and radius)

**3D accumulator**: $A(a, b, r)$
- More computationally expensive
- OpenCV uses gradient information to reduce complexity

In [ ]:
# Hough Line Transform
print("=" * 70)
print("HOUGH LINE TRANSFORM")
print("=" * 70)

# Create image with lines
img_lines = np.zeros((500, 500), dtype=np.uint8)
cv2.line(img_lines, (100, 100), (400, 150), 255, 2)
cv2.line(img_lines, (50, 300), (450, 350), 255, 2)
cv2.line(img_lines, (200, 50), (250, 450), 255, 2)
cv2.line(img_lines, (350, 100), (400, 400), 255, 2)

# Edge detection
edges = cv2.Canny(img_lines, 50, 150)

# Standard Hough Line Transform
lines_standard = cv2.HoughLines(edges, 1, np.pi/180, 100)

img_hough_standard = cv2.cvtColor(img_lines, cv2.COLOR_GRAY2BGR)
if lines_standard is not None:
    print(f"\nStandard Hough: {len(lines_standard)} lines detected")
    for line in lines_standard:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(img_hough_standard, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Probabilistic Hough Line Transform
lines_prob = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=10)

img_hough_prob = cv2.cvtColor(img_lines, cv2.COLOR_GRAY2BGR)
if lines_prob is not None:
    print(f"Probabilistic Hough: {len(lines_prob)} line segments detected")
    for line in lines_prob:
        x1, y1, x2, y2 = line[0]
        cv2.line(img_hough_prob, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img_lines, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_hough_standard, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Standard Hough\n{len(lines_standard) if lines_standard is not None else 0} lines (Red)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_hough_prob, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Probabilistic Hough\n{len(lines_prob) if lines_prob is not None else 0} segments (Green)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Hough Transform finds lines even in noisy images!")

In [ ]:
# Hough Circle Transform
print("=" * 70)
print("HOUGH CIRCLE TRANSFORM")
print("=" * 70)

# Create image with circles
img_circles = np.zeros((500, 500), dtype=np.uint8)
cv2.circle(img_circles, (150, 150), 50, 255, 2)
cv2.circle(img_circles, (350, 150), 70, 255, 2)
cv2.circle(img_circles, (150, 350), 40, 255, 2)
cv2.circle(img_circles, (350, 350), 60, 255, 2)

# Apply Hough Circle Transform
circles = cv2.HoughCircles(img_circles, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                           param1=50, param2=30, minRadius=20, maxRadius=100)

img_detected = cv2.cvtColor(img_circles, cv2.COLOR_GRAY2BGR)

if circles is not None:
    circles = np.uint16(np.around(circles))
    print(f"\nCircles detected: {len(circles[0])}\n")
    
    for i, (x, y, r) in enumerate(circles[0]):
        print(f"Circle {i}: center=({x}, {y}), radius={r}")
        # Draw circle
        cv2.circle(img_detected, (x, y), r, (0, 255, 0), 3)
        # Draw center
        cv2.circle(img_detected, (x, y), 2, (0, 0, 255), 3)
else:
    print("\nNo circles detected")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(img_circles, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_detected, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Detected Circles\n{len(circles[0]) if circles is not None else 0} circles (Green)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Hough Circle Transform finds circles of various sizes!")

## Summary

### Key Concepts

1. **Contours**: Boundary representation of shapes
2. **Moments**: Quantitative shape descriptors
3. **Bounding Shapes**: Simplified enclosing shapes
4. **Approximation**: Simplify contours (Douglas-Peucker)
5. **Hough Transform**: Detect parametric shapes

### Shape Properties Summary

| Property | Formula | Meaning |
|----------|---------|----------|
| Area | $M_{00}$ or `contourArea()` | Total pixels |
| Perimeter | `arcLength(contour, True)` | Boundary length |
| Centroid | $(M_{10}/M_{00}, M_{01}/M_{00})$ | Center of mass |
| Compactness | $4\pi A / P^2$ | Circularity (1.0 = circle) |
| Extent | $A_{\text{contour}} / A_{\text{rect}}$ | Fullness of bounding box |
| Solidity | $A_{\text{contour}} / A_{\text{hull}}$ | Convexity measure |

### Mathematical Formulas Reference

**Moments**:
$$
M_{ij} = \sum_x \sum_y x^i y^j I(x,y)
$$

**Centroid**:
$$
\bar{x} = \frac{M_{10}}{M_{00}}, \quad \bar{y} = \frac{M_{01}}{M_{00}}
$$

**Hough Line**:
$$
\rho = x\cos\theta + y\sin\theta
$$

**Hough Circle**:
$$
(x-a)^2 + (y-b)^2 = r^2
$$

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Find contours | `cv2.findContours(img, mode, method)` | RETR_EXTERNAL, CHAIN_APPROX_SIMPLE |
| Draw contours | `cv2.drawContours(img, contours, idx, color)` | idx=-1 for all |
| Moments | `cv2.moments(contour)` | Returns dict with M_ij |
| Area | `cv2.contourArea(contour)` | Oriented area |
| Perimeter | `cv2.arcLength(contour, closed)` | closed=True for loops |
| Bounding rect | `cv2.boundingRect(contour)` | Returns (x,y,w,h) |
| Min area rect | `cv2.minAreaRect(contour)` | Rotated rectangle |
| Enclosing circle | `cv2.minEnclosingCircle(contour)` | Returns center, radius |
| Approximate | `cv2.approxPolyDP(contour, epsilon, closed)` | Douglas-Peucker |
| Convex hull | `cv2.convexHull(contour)` | Smallest convex set |
| Hough lines | `cv2.HoughLines(edges, rho, theta, threshold)` | Standard |
| Hough lines P | `cv2.HoughLinesP(edges, ...)` | Probabilistic |
| Hough circles | `cv2.HoughCircles(img, method, dp, minDist)` | HOUGH_GRADIENT |

**Next**: Module 8 - Histograms